In [1]:
import pandas as pd
import openai
import os
import json
import time

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
transcript = f"""
Participant 1: "Animal testing is necessary for medical research. It has contributed to numerous medical advancements and the 
development of life-saving treatments." 
Participant 2: "While I understand the importance of medical research, 
we should also consider the ethical concerns associated with animal testing. Animals deserve to be treated with 
compassion and should not suffer for the sake of human benefits." 
Participant 3: "There are alternatives to animal testing, such as in vitro testing and computer simulations, that can provide reliable results without causing harm 
to animals. We should prioritize the development and adoption of these alternatives." 
Participant 4: "Animal testing should be regulated and minimized, but we cannot completely eliminate it at this stage. Striking a balance between scientific 
progress and animal welfare is crucial." 
Participant 5: "We need more transparency and accountability in animal testing.
Researchers should provide clear justifications for using animals and ensure that it is conducted in the most humane way 
possible.

<br>Participant 1: "The role of social media in political campaigns is a subject that has gained significant attention in recent 
years. It has become a powerful tool for politicians to engage with voters and spread their message. 
However, there are concerns about the spread of misinformation and the manipulation of public opinion through targeted ads." 
<br>Participant 2: "I agree that social media provides a platform for political candidates to connect with a wider audience and 
mobilize support. However, the lack of regulation and transparency in political advertising on these platforms is a major 
issue that needs to be addressed." 
<br>Participant 3: "I believe that social media has democratized political discourse and 
allowed marginalized voices to be heard. It provides a platform for grassroots movements and enables citizens to 
participate in political discussions like never before. We should focus on educating users about media literacy and critical 
thinking to combat misinformation." 
<br>Participant 4: "While social media has its benefits, the algorithms used by these platforms tend to create echo chambers and 
reinforce existing biases. We need better regulation to ensure that diverse viewpoints are represented and to prevent the 
manipulation of public opinion through targeted content.

<br>Speaker 1: "We've seen a rise in youth violence in our community, and I'm worried. Maybe some sort of mentoring program 
could make a difference?" 
<br>Speaker 2: "Mentoring? Really? In my opinion, we need more police presence. That's the only language these kids understand." 
<br>Speaker 3: "I wouldn't discount the value of a mentoring program so quickly. The problem isn't just law enforcement, but also 
social issues. Addressing those might help." 
<br>Speaker 4: "If you ask me, this is a problem that begins at home. Many of these kids lack a good family structure. We need 
programs that support families too." 
<br>Speaker 5: "All these are good points, but what about the role of schools? They're underfunded and struggling to offer good 
education and extracurricular activities." 
<br>Speaker 6: "Yes, and we also have to remember the role of peer pressure in youth violence. We need programs that teach our 
"""

In [4]:
transcript_2 = f"""
Participant 1: "Animal testing is necessary for medical research. It has contributed to numerous medical advancements and the 
development of life-saving treatments." 
Participant 2: "While I understand the importance of medical research, 
we should also consider the ethical concerns associated with animal testing. Animals deserve to be treated with 
compassion and should not suffer for the sake of human benefits." 
Participant 3: "There are alternatives to animal testing, such as in vitro testing and computer simulations, that can provide reliable results without causing harm 
to animals. We should prioritize the development and adoption of these alternatives." 
Participant 4: "Animal testing should be regulated and minimized, but we cannot completely eliminate it at this stage. Striking a balance between scientific 
progress and animal welfare is crucial." 
Participant 5: "We need more transparency and accountability in animal testing.
Researchers should provide clear justifications for using animals and ensure that it is conducted in the most humane way 
possible.
"""

In [5]:
transcript_3 = f"""
<br>Speaker 1: "We've seen a rise in youth violence in our community, and I'm worried. Maybe some sort of mentoring program 
could make a difference?" 
<br>Speaker 2: "Mentoring? Really? In my opinion, we need more police presence. That's the only language these kids understand." 
<br>Speaker 3: "I wouldn't discount the value of a mentoring program so quickly. The problem isn't just law enforcement, but also 
social issues. Addressing those might help." 
<br>Speaker 4: "If you ask me, this is a problem that begins at home. Many of these kids lack a good family structure. We need 
programs that support families too." 
<br>Speaker 5: "All these are good points, but what about the role of schools? They're underfunded and struggling to offer good 
education and extracurricular activities." 
<br>Speaker 6: "Yes, and we also have to remember the role of peer pressure in youth violence. We need programs that teach our 
"""

In [6]:
transcript_4 = f"""
P1: "Have you all been following the news about the rising sea levels? Our coastal cities are under threat and we can't just 
sit and watch. We need to be proactive in our mitigation efforts, perhaps even consider relocating our operations to a safer, 
inland location." P2: "I share your concerns, P1, but relocating seems a bit extreme, doesn't it? It's a costly measure. I 
think we should focus on increasing our preparedness for extreme weather events. Let's reinforce our infrastructure, have 
contingency plans in place. Better safe than sorry, right?" P3: "While I appreciate both points, I must stress that we can't 
tackle this issue single-handedly. We need to lobby for more robust government measures. It's a national issue." 
P4: "Indeed, P3. But we also have a responsibility to reduce our own carbon footprint. Maybe, investing in green 
technologies or shifting to cleaner production methods?" P5: "I like that idea, P4. In fact, it might even prove to be 
beneficial for our brand image. We can portray ourselves as a responsible, environmentally-conscious corporation." 
P6: "All points considered, we need to come up with a comprehensive plan. Let's get our teams on this - risk management, 
PR, everyone. And let's engage with local communities, they need to be part of this conversation too."
"""

#### Helper function to get completions from model based on a list of prompts; model to be called can be changed (e.g., GPT-3.5-turbo instead of GPT-4)


In [7]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

#### Function for getting completions from OpenAI instructing model to identify topics based on a discussion transcript

In [8]:
def get_topics(transcript):
    prompt_topic = f"""
    You are an assistant for group discussions, specializing in keeping track of and documenting the discussion,
that is, the topics discussed, the viewpoints/positions on each topic, and the arguments/explanations given in support of each viewpoint.

Identify the main topic discussed in the discussion transcript, delimited with triple backticks.

Check the following steps but don't print each step, only print what is asked:

Step 1: Identify the main topic discussed in the discussion transcript. Take the time to read the complete transcript and 
        understand it, don't take words as a topic, analyze the arguments in the transcript and then decide on the topic. \
Step 2: Analyze each topic (if there is more than one) and merge those into one main topic.
        If there are topics that are as important as the main topic, display them as secondary topics, but only if they are 
        closely linked to the main topic of discussion in the transcript. \
Step 3: State the necessary main topic of the entire transcript in a concise, descriptive sentence,
        in up to 3 words for each topic. Consider the frequency of mentions, relevance, and significance of each topic.
        Discard any topic that is not at least 50% as relevant as the main topic. \
Step 4: Review the main topic and the transcript again. If any secondary topics are not closely related to the main topic,
        discard them and print only the main topic. \
Step 5: Provide the output in a JSON format where the key is each topic and the values are the text 'viewpoints'. Don't write
the text 'main topic' and don't display secondary topics.
        for example:
            {{main topic: 'viewpoint'}}

    Review transcript: '''{transcript}'''
    """

    topics = get_completion(prompt_topic)
    topics_json = json.loads(topics)

    return topics_json

In [9]:
def process_transcript(transcript):
    start_time = time.time()  # Start measuring time
    topics_test = get_topics(transcript)
    end_time = time.time()  # Stop measuring time
    execution_time = end_time - start_time  # Calculate the execution time
    return {"topics": topics_test, "execution_time": execution_time}

In [16]:
topics = process_transcript(transcript_4)
print(topics)

{'topics': {'Rising Sea Levels': 'viewpoint'}, 'execution_time': 2.6916348934173584}


In [24]:
def get_viewpoints_by_topic(topics,transcript):
    result = {}
    for topic, excerpt in topics.items():
        prompt = f"""
        You are an assistant for group discussions, specialized on keeping track and documenting the discussion,
        that is, the topics discussed, the viewpoints/positions on each topic and the arguments/explanations given in support of each viewpoint.
        For the topic '{topic}'elimited by triple hashtags, analyse the corresponding excerpt from the discussion.
        Your task is to identify all the viewpoints expressed on the topic.

        Proceed according to the following steps:

        Step 1: In the below discussion transcript, delimited by triple backticks,
                locate the excerpt corresponding to the topic and consider the rest
                of the transcript as context for the subsequent steps.
        Step 2: Are there one or several viewpoints being expressed in the excerpt?
                A "viewpoint" refers to "one's perspective of opinion on a particular topic".
        Step 3: If there is only one viewpoint, summarize the viewpoint in 3 keywords max,
                more keywords only if necessary to fully grasp the viewpoint. Viewpoint keywords
                should be expressed as noun phrases that describe the viewpoint in a depersonalized manner.
                For example, instead of “Supports Renewables”, the viewpoint keyword should be “Support for Renewables”.
                Instead of “Believes in Traditional Energy”, the viewpoint keyword should be “Belief in Traditional Energy”.

                If there are several viewpoints, summarize each viewpoint in 3 keywords max, more keywords only if
                necessary to fully grasp the topic. Viewpoint keywords should be expressed as noun phrases that describe
                the viewpoint in a depersonalized manner, as explained in the instruction for one viewpoint.
        Step 4: Disregard viewpoints that are not relevant to the current topic or more relevant to another topic. Only if a viewpoint is equally relevant to multiple topics, include it under all relevant topics.
        Step 5: Identify any linkages between viewpoints that build upon each other or propose solutions to identified issues. For instance, if a viewpoint such as 'Media literacy and critical thinking' is expressed as a solution to the issue identified in another viewpoint like 'Concerns about misinformation', classify it as a sub-viewpoint of the latter. Represent these sub-viewpoints appropriately within the hierarchical structure of the result dictionary.
        Step 6: Identify viewpoints that convey essentially the same stance on the topic. For example, viewpoints like 'Lack of regulation and transparency' and 'Need for better regulation' express similar concerns regarding the need for increased regulation in the domain. In such cases, merge these viewpoints into a single unified viewpoint that encapsulates both perspectives. Ensure this is reflected in the summary of viewpoints in the result dictionary.
        Step 7: Append the topics with the identified viewpoints as sub-keys. Format the dictionary in JSON-format.

        Only include the appended dictionary in your response.

        ```{transcript}```
        """
        viewpoints = get_completion(prompt)
        viewpoints_json = json.loads(viewpoints)
        result.update(viewpoints_json)
    return result

In [25]:
start_time = time.time()
result_view = get_viewpoints_by_topic(topics, transcript_4)
execution_time = time.time() - start_time

# Print the result and execution time
print(result_view)
print("Execution Time:", execution_time, "seconds")

{'Rising Sea Levels': {'Relocating Operations': {}, 'Increasing Preparedness': {}, 'Lobbying for Government Measures': {}, 'Reducing Carbon Footprint': {'Investing in Green Technologies': {}, 'Cleaner Production Methods': {}}, 'Improving Brand Image': {}, 'Comprehensive Plan': {'Risk Management': {}, 'Public Relations': {}, 'Community Engagement': {}}}, 'execution_time': {'Relocation as mitigation': {}, 'Preparedness for extreme weather': {}, 'Lobbying for government measures': {}, 'Reducing carbon footprint': {}, 'Brand image benefits': {}, 'Comprehensive plan': {'Involving multiple teams': {}, 'Engaging local communities': {}}}}
Execution Time: 33.50298309326172 seconds


In [ ]:
def get_arguments_by_viewpoint(viewpoints_by_topic):
    result = {}
    for topic, viewpoints in viewpoints_by_topic.items():
        arguments = {}
        for viewpoint, summary in viewpoints.items():
            prompt = f"""
            You are an assistant for group discussions, specialized in keeping track and documenting the discussion,
            that is, the topics discussed, the viewpoints/positions on each topic, and the arguments/explanations given in support of each viewpoint.
            For the topic '{topic}' and viewpoint '{viewpoint}', you need to provide the arguments/explanations
            given in support of this viewpoint.

            Proceed according to the following steps:

            Step 1: Review the transcript excerpt related to the viewpoint '{viewpoint}' on the topic '{topic}'.
            Step 2: Identify the arguments or explanations provided in support of this viewpoint.
            Step 3: Summarize the arguments/explanations in one to two sentences.

            Excerpt:
            {summary}
            """

            arguments_completion = get_completion(prompt)
            arguments_json = json.loads(arguments_completion)
            arguments[viewpoint] = arguments_json

        result[topic] = arguments

    return result

In [ ]:
def process_transcript(transcript):
    topics = get_topics(transcript)
    viewpoints = get_viewpoints_by_topic(topics, transcript)
    arguments = get_arguments_by_viewpoint(viewpoints)
    return {"topics": topics, "viewpoints": viewpoints, "arguments": arguments}

In [9]:
def process_transcript_batch(transcripts):
    results = []
    for transcript in transcripts:
        result = process_transcript(transcript)
        results.append(result)
    return results

In [12]:
result = process_transcript(transcript)
print(result)

RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fa7e56834b8a406262a344005a49f257 in your message.)